In [1]:
import torch
import numpy as np
import pandas as pd
from Custom_dataset import CustomDataset

In [2]:
adult_train = pd.read_csv('./dataset/ECG_adult_age_train.csv')

In [3]:
file_path = './dataset/ECG_adult_numpy_train/'

adult_dataset = CustomDataset(file_path,adult_train)

In [4]:
adult_dataset[0]

TypeError: array_to_dict_by_name() takes 1 positional argument but 2 were given